In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/Users/fdangelo/PycharmProjects/myRBM")
import numpy as np 
import scipy.constants as cs
import deepdish as dd
from sklearn.preprocessing import Binarizer
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
data = dd.io.load('/Users/fdangelo/PycharmProjects/myRBM/data/ising/ising_data.hdf5')
data.keys()

dict_keys(['T=2.000000', 'T=2.269000', 'T=2.500000'])

In [3]:
data_T2 = data['T=2.000000']
data_T2_26 = data['T=2.269000']
data_T2_5 = data['T=2.500000']

In [4]:
binarizer = Binarizer(threshold=0)
data_T2_binary = np.array([binarizer.fit_transform(slice) for slice in data_T2])
data_T2_26_binary = np.array([binarizer.fit_transform(slice) for slice in data_T2_26])
data_T2_5_binary = np.array([binarizer.fit_transform(slice) for slice in data_T2_5])

In [5]:
magn_T2 = np.abs(np.array([np.average(slice) for slice in data_T2]))
magn_T2_26 = np.abs(np.array([np.average(slice) for slice in data_T2_26]))
magn_T2_5= np.abs(np.array([np.average(slice) for slice in data_T2_5]))

In [21]:
# create a new plot with default tools, using figure
p = figure(plot_width=600, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(np.arange(len(magn_T2)),magn_T2, size=7, line_color="navy", fill_color="green", fill_alpha=0.5, legend = 'T = 2.0')
p.circle(np.arange(len(magn_T2)),magn_T2_26, size=7, line_color="navy", fill_color="yellow", fill_alpha=0.5, legend = 'T = 2.269')
p.circle(np.arange(len(magn_T2)),magn_T2_5, size=7, line_color="navy", fill_color="blue", fill_alpha=0.5, legend = 'T = 2.5')

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "Sample"
p.legend.location = "bottom_right"
show(p) # show the results

In [9]:
mean_magn_T2 = np.average(magn_T2)
mean_magn_T2_26 = np.average(magn_T2_26)
mean_magn_T2_5= np.average(magn_T2_5)
std_magn_T2 = np.std(magn_T2)
std_magn_T2_26 = np.std(magn_T2_26)
std_magn_T2_5= np.std(magn_T2_5)
mean_magn_temp = [mean_magn_T2,mean_magn_T2_26,mean_magn_T2_5]
std_magn_temp = [std_magn_T2,std_magn_T2_26,std_magn_T2_5]
temp = [2.0/2.269,2.269/2.269,2.5/2.269]
T = np.linspace(0.1,2.3,100)
teor_magn = np.nan_to_num(np.power((1-np.power(np.sinh(2/(T)),-4)),0.125))

/Users/fdangelo/anaconda3/envs/RBM/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in power
  # This is added back by InteractiveShellApp.init_path()


In [22]:
# create a new plot with default tools, using figure
from bokeh.models import ColumnDataSource, Whisker
p = figure(plot_width=600, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(temp,mean_magn_temp, size=7, line_color="navy", fill_color="green", fill_alpha=0.5, legend = 'Simulated conf')
p.line(T/2.269, teor_magn,line_color="red", line_width=3, line_alpha=0.6, legend='Theoretical curve' )
source_error = ColumnDataSource(data=dict(base=temp, lower=np.asarray(mean_magn_temp)-np.asarray(std_magn_temp), upper=np.asarray(mean_magn_temp)+np.asarray(std_magn_temp)))
p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

p.yaxis.axis_label = "Magnetization"
p.xaxis.axis_label = "T/Tcritic"
p.legend.location = "bottom_left"
show(p) # show the results